In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import time
import os
from itertools import product
from pylab import rcParams
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_log_error
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess, Fourier
path = 'data/'
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
oil = pd.read_csv(path + 'oil.csv')
holidays = pd.read_csv(path + 'holidays_events.csv')
stores = pd.read_csv(path + 'stores.csv')
transactions = pd.read_csv(path + 'transactions.csv')

##### Data Manipulation

Calender from 2013 to 2017

In [3]:
calendar = pd.DataFrame(index=pd.date_range('2013-01-01', '2017-08-31'))

# moving average
df_oil = pd.read_csv(path + 'oil.csv', parse_dates=['date'], infer_datetime_format=True, index_col='date')
df_oil['ma_oil'] = df_oil['dcoilwtico'].rolling(7).mean()

calendar = calendar.merge(df_oil, how='left', left_index=True, right_index=True)
calendar['ma_oil'].fillna(method='ffill', inplace=True)

In [4]:
# Day of week
calendar['dofw'] = calendar.index.dayofweek

In [5]:
calendar

,dcoilwtico,ma_oil,dofw
2013-01-01,NaN,NaN,1
2013-01-02,93.14,NaN,2
2013-01-03,92.97,NaN,3
2013-01-04,93.12,NaN,4
2013-01-05,NaN,NaN,5
...,...,...,...
2017-08-27,NaN,47.720000,6
2017-08-28,46.40,47.624286,0
2017-08-29,46.46,47.320000,1
2017-08-30,45.96,47.115714,2


Holiday events

In [6]:
df_events = pd.read_csv(path + 'holidays_events.csv', parse_dates=['date'], infer_datetime_format=True)
df_events['date'] = df_events['date'].replace({'2013-04-29' : 
                                         pd.to_datetime('2013-03-29')}) # 'Good Friday' mistake correction

df_events = df_events.set_index('date').sort_index()
df_events = df_events[df_events.locale == 'National'] 
df_events = df_events.groupby(df_events.index).first() # Keep one event only

In [7]:
# Including workday to the calender 
calendar['wd'] = True

calendar.loc[calendar.dofw > 4, 'wd'] = False
calendar = calendar.merge(df_events, how='left', left_index=True, right_index=True)
calendar.loc[calendar.type == 'Bridge'  , 'wd'] = False
calendar.loc[calendar.type == 'Work Day', 'wd'] = True
calendar.loc[calendar.type == 'Transfer', 'wd'] = False
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == False), 'wd'] = False
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == True ), 'wd'] = True

In [8]:
calendar

,dcoilwtico,ma_oil,dofw,wd,type,locale,locale_name,description,transferred
2013-01-01,NaN,NaN,1,False,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-02,93.14,NaN,2,True,NaN,NaN,NaN,NaN,NaN
2013-01-03,92.97,NaN,3,True,NaN,NaN,NaN,NaN,NaN
2013-01-04,93.12,NaN,4,True,NaN,NaN,NaN,NaN,NaN
2013-01-05,NaN,NaN,5,True,Work Day,National,Ecuador,Recupero puente Navidad,False
...,...,...,...,...,...,...,...,...,...
2017-08-27,NaN,47.720000,6,False,NaN,NaN,NaN,NaN,NaN
2017-08-28,46.40,47.624286,0,True,NaN,NaN,NaN,NaN,NaN
2017-08-29,46.46,47.320000,1,True,NaN,NaN,NaN,NaN,NaN
2017-08-30,45.96,47.115714,2,True,NaN,NaN,NaN,NaN,NaN


##### Data Modeling

Rigde Regression

In [9]:
df_train = pd.read_csv(path + 'train.csv',
                       usecols=['store_nbr', 'family', 'date', 'sales'],
                       dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float32'},
                       parse_dates=['date'], infer_datetime_format=True)
df_train.date = df_train.date.dt.to_period('D')
df_train = df_train.set_index(['store_nbr', 'family', 'date']).sort_index()

df_test = pd.read_csv(path + 'test.csv',
                      usecols=['store_nbr', 'family', 'date'],
                      dtype={'store_nbr': 'category', 'family': 'category'},
                      parse_dates=['date'], infer_datetime_format=True)
df_test.date = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

In [10]:
df_train

sales
store_nbr family     date                 
1         AUTOMOTIVE 2013-01-01   0.000000
                     2013-01-02   2.000000
                     2013-01-03   3.000000
                     2013-01-04   3.000000
                     2013-01-05   5.000000
...                                    ...
9         SEAFOOD    2017-08-11  23.830999
                     2017-08-12  16.859001
                     2017-08-13  20.000000
                     2017-08-14  17.000000
                     2017-08-15  16.000000

[3000888 rows x 1 columns]

In [11]:
end_date='2017-08-15'
start_date='2017-04-01'

In [12]:
y = df_train.unstack(['store_nbr', 'family']).loc[start_date:end_date]

fourier = CalendarFourier(freq='W', order=4)
dp = DeterministicProcess(index=y.index,
                          constant=False,
                          order=1,
                          seasonal=False,
                          additional_terms=[fourier],
                          drop=True)
X = dp.in_sample()

# Extentions

X['oil']  = calendar.loc[start_date:end_date]['ma_oil'].values
X['dofw'] = calendar.loc[start_date:end_date]['dofw'].values
X['wd']   = calendar.loc[start_date:end_date]['wd'].values
X['type'] = calendar.loc[start_date:end_date]['type'].values

X = pd.get_dummies(X, columns=['dofw'], drop_first=True)
X = pd.get_dummies(X, columns=['type'], drop_first=False)

model = Ridge(fit_intercept=True, solver='auto', alpha=0.5, normalize=True) # try alpha,0.1 ,0.3 ,0.6 and 0.9
model.fit(X, y)
y_pred = pd.DataFrame(model.predict(X), index=X.index, columns=y.columns)

In [13]:
y_pred   = y_pred.stack(['store_nbr', 'family']).reset_index()
y_target = y.stack(['store_nbr', 'family']).reset_index().copy()
y_target['sales_pred'] = y_pred['sales'].clip(0.) 
y_target.groupby('family').apply(lambda r: mean_squared_log_error(r['sales'], r['sales_pred']))

family
AUTOMOTIVE                    0.259202
BABY CARE                     0.066660
BEAUTY                        0.267450
BEVERAGES                     0.199187
BOOKS                         0.026701
BREAD/BAKERY                  0.125449
CELEBRATION                   0.295910
CLEANING                      0.204513
DAIRY                         0.136196
DELI                          0.108830
EGGS                          0.147672
FROZEN FOODS                  0.145027
GROCERY I                     0.210306
GROCERY II                    0.347753
HARDWARE                      0.273930
HOME AND KITCHEN I            0.259483
HOME AND KITCHEN II           0.219104
HOME APPLIANCES               0.154737
HOME CARE                     0.122103
LADIESWEAR                    0.259483
LAWN AND GARDEN               0.216374
LINGERIE                      0.400216
LIQUOR,WINE,BEER              0.612719
MAGAZINES                     0.254086
MEATS                         0.123141
PERSONAL CARE     

RandomForest and Ridge Regression

In [14]:
from joblib import Parallel, delayed
import warnings

from sklearn.linear_model import Ridge
from sklearn.ensemble     import RandomForestRegressor

class CustomRegressor():
    
    def __init__(self, n_jobs=-1, verbose=0): 
        self.n_jobs = n_jobs
        self.verbose = verbose
        self.estimators_ = None
        
    def _estimator_(self, X, y):
        warnings.simplefilter(action='ignore', category=FutureWarning)
        if y.name[2] == 'SCHOOL AND OFFICE SUPPLIES':
            model = RandomForestRegressor(n_estimators = 300, n_jobs=-1, random_state=1)
            
        else:
            model = Ridge(fit_intercept=True, solver='auto', alpha=0.5, normalize=True)
        model.fit(X, y)
        return model

    def fit(self, X, y):
        self.estimators_ = Parallel(n_jobs=self.n_jobs, 
                              verbose=self.verbose,
                              )(delayed(self._estimator_)(X, y.iloc[:, i]) for i in range(y.shape[1]))
        return
    
    def predict(self, X):
        y_pred = Parallel(n_jobs=self.n_jobs, 
                          verbose=self.verbose)(delayed(e.predict)(X) for e in self.estimators_)
        return np.stack(y_pred, axis=1)

In [15]:
model = CustomRegressor(n_jobs=-1, verbose=0)
model.fit(X, y)
y_pred = pd.DataFrame(model.predict(X), index=X.index, columns=y.columns)

In [16]:
# RandomForest Model results
y_pred   = y_pred.stack(['store_nbr', 'family']).reset_index()
y_target = y.stack(['store_nbr', 'family']).reset_index().copy()
y_target['sales_pred'] = y_pred['sales'].clip(0.) 
y_target.groupby('family').apply(lambda r: mean_squared_log_error(r['sales'], r['sales_pred']))

family
AUTOMOTIVE                    0.259202
BABY CARE                     0.066660
BEAUTY                        0.267450
BEVERAGES                     0.199187
BOOKS                         0.026701
BREAD/BAKERY                  0.125449
CELEBRATION                   0.295910
CLEANING                      0.204513
DAIRY                         0.136196
DELI                          0.108830
EGGS                          0.147672
FROZEN FOODS                  0.145027
GROCERY I                     0.210306
GROCERY II                    0.347753
HARDWARE                      0.273930
HOME AND KITCHEN I            0.259483
HOME AND KITCHEN II           0.219104
HOME APPLIANCES               0.154737
HOME CARE                     0.122103
LADIESWEAR                    0.259483
LAWN AND GARDEN               0.216374
LINGERIE                      0.400216
LIQUOR,WINE,BEER              0.612719
MAGAZINES                     0.254086
MEATS                         0.123141
PERSONAL CARE     

In [17]:
end_test='2017-08-31'
start_test='2017-08-16'
X_test = dp.out_of_sample(steps=16)

# Extentions
X_test['oil']  = calendar.loc[start_test:end_test]['ma_oil'].values
X_test['dofw'] = calendar.loc[start_test:end_test]['dofw'].values
X_test['wd']   = calendar.loc[start_test:end_test]['wd'].values

X_test = pd.get_dummies(X_test, columns=['dofw'], drop_first=True)

# No national level events in this period
X_test[['type_Additional', 'type_Event', 'type_Holiday', 'type_Transfer']] = 0

sales_pred = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)
sales_pred = sales_pred.stack(['store_nbr', 'family'])
sales_pred[sales_pred < 0] = 0. 

In [18]:
My_submission = pd.read_csv(path + 'sample_submission.csv', index_col='id')
My_submission.sales = sales_pred.values
# My_submission.to_csv('R_model.csv', index=True)

##### Kaggle RMSLE: 0.42